In [41]:
import pandas as pd
from datetime import datetime
import re

from langdetect import detect
import os

ModuleNotFoundError: No module named 'langdetect'

## Analizamos resultados

Vemos que curiosamente la `empresa` y la `duración de jornada` son campos que están vacíos en casi todas las ofertas

In [16]:
def read_file(path):
    df = pd.read_csv(path)#("ofertas/ofertas11_092659.csv")
    filasNulas = df[df.isna().sum(axis="columns") >= df.shape[1]-1]

    print(f"El dataset tiene {df.shape[0]} ofertas, de las cuales, {filasNulas.shape[0]} están vacías (excepto la url)")
    print("\nNúmero de nulos por columna, excluyendo filas completamente nulas:")
    print(df.isna().sum()[df.isna().sum() > filasNulas.shape[0]] - filasNulas.shape[0])

    return df, df.drop(filasNulas.index)

In [14]:
path = "ofertasFast/ofertas11_180313.csv"
df, clean = read_file(path)
clean.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ofertasFast/ofertas11_180313.csv'

In [4]:
clean["fecha_publicacion"].unique()

array(['11/10/2025', '08/10/2025', '10/10/2025'], dtype=object)

In [ ]:
clean["fecha_limite"].unique()

array(['16/09/2025', '19/11/2025', '15/10/2025', '31/10/2025',
       '29/10/2025', '16/10/2025', '15/11/2025', '19/10/2025',
       '21/11/2025', '16/11/2025', '25/10/2025', '30/10/2025',
       '23/10/2025', '10/10/2025', '11/10/2025', '17/11/2025',
       '20/10/2025', '18/10/2025', '24/11/2025', '31/12/2025',
       '22/10/2025', '27/10/2025', '24/10/2025', '21/10/2025',
       '25/11/2025', '18/11/2025', '20/11/2025', '14/10/2025',
       '7/11/2025', nan, '11/11/2025', '26/11/2025', '17/10/2025',
       '26/10/2025', '27/11/2025', '29/11/2025', '1/01/2026',
       '23/11/2025', '28/11/2025', '29/12/2025', '13/10/2025',
       '28/10/2025', '26/10/2026', '1/12/2025', '2/11/2025', '30/12/2025',
       '2/12/2025', '1/11/2025', '3/11/2025', '12/10/2025', '5/12/2025',
       '3/12/2025', '4/12/2025', '4/11/2025', '30/11/2025', '7/12/2025',
       '8/12/2025', '11/12/2025', '9/12/2025', '8/11/2025', '10/12/2025',
       '10/11/2025', '9/11/2025', '6/11/2025', '30/01/2026', '12/12/2025

## Cargamos ambos, filtramos por español y guardamos en uno común

In [31]:
print("Clean1:")
_, clean1 = read_file("ofertas/ofertas11_092659.csv")
print("\nClean2:")
_, clean2 = read_file("ofertasFast/ofertas11_180313.csv")
clean = pd.concat([clean1, clean2], ignore_index=True)

Clean1:
El dataset tiene 3971 ofertas, de las cuales, 221 están vacías (excepto la url)

Número de nulos por columna, excluyendo filas completamente nulas:
fecha_limite          23
empresa             3642
ocupacion            363
region               152
duracion_jornada    3686
tipo_contrato          4
dtype: int64

Clean2:
El dataset tiene 6440 ofertas, de las cuales, 505 están vacías (excepto la url)

Número de nulos por columna, excluyendo filas completamente nulas:
fecha_limite          11
empresa             5831
ocupacion            576
region               170
duracion_jornada    5934
tipo_contrato          4
dtype: int64


In [34]:
clean_es = clean[clean['descripcion'].apply(lambda r: detect(r) == 'es')]
clean.shape, clean_es.shape

((9685, 12), (7435, 12))

In [35]:
clean_es.to_csv("ofertasRaw.csv")

## Gestionar valores duplicados y NaN

In [17]:
clean = pd.read_csv("ofertasRaw.csv").drop(["Unnamed: 0"], axis="columns")
clean.drop_duplicates(subset=['url'], inplace=True)
clean.duplicated().sum()

np.int64(0)

In [18]:
clean.set_index("url")

,fecha_publicacion,fecha_limite,titulo,empresa,ocupacion,educacion,descripcion,pais,region,duracion_jornada,tipo_contrato
url,,,,,,,,,,,
https://europa.eu/eures/portal/jv-se/jv-details/NzE4NDAzMCAxMDI?lang=es,10/10/2025,31/10/2025,"CONDUCTORES DE CAMIÓN, EN GENERAL",NaN,Conductor de vehículo de carga/conductora de v...,Contrato,REQUIERE ESTAR EN POSESIÓN DEL CAP. LA EMPRESA...,España,Palencia -,NaN,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyMDEwMCAxMDI?lang=es,10/10/2025,10/10/2025,WORK/LOCATION,NaN,Empleado administrativo de archivos/empleada a...,Contrato,ES BUSCA UNA PERSONA AMB ESTUDIS I EXPERIèNCIA...,España,Tarragona -,NaN,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyNzY1MCAxMDI?lang=es,10/10/2025,24/10/2025,WORK/LOCATION,NaN,Operario especializado en engranajes de automo...,Contrato,CERQUEM PROFESSIONALS AMB EXPERIèNCIA EN MUNTA...,España,Girona -,NaN,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyMDg4OSAxMDI?lang=es,10/10/2025,13/10/2025,WORK/LOCATION,NaN,Empleado del hogar/empleada del hogar,Contrato,IMPORTANTE EMPRESA UBICADA EN MERCABARNA PRECI...,España,Barcelona -,NaN,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyNDgzOSAxMDI?lang=es,10/10/2025,20/10/2025,WORK/LOCATION,NaN,Conductor de servicios de transporte de enferm...,Determinado,REQUISITS: ? TITULACIÓ D? ESPECIALISTA EN FARM...,España,Barcelona -,NaN,Determinado
...,...,...,...,...,...,...,...,...,...,...,...
https://europa.eu/eures/portal/jv-se/jv-details/NzIzOTcwNyAxMDI?lang=es,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Inspector de carga y flete/inspectora de carga...,Determinado,"ACTUALMENTE, DESDE LA DELEGACIÓN DE TARRAGONA ...",España,Barcelona -,NaN,Determinado
https://europa.eu/eures/portal/jv-se/jv-details/NzIzOTcxNiAxMDI?lang=es,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Profesor de autoescuela/profesora de autoescuela,Autónomo,"NDICE CONSULTORÍA Y FORMACIÓN, SOMOS UNA CONSU...",España,Barcelona -,NaN,Autónomo
https://europa.eu/eures/portal/jv-se/jv-details/NzIzOTczMiAxMDI?lang=es,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Mayordomo del servicio doméstico,Contrato,HELPOINT -ÉS UNA CONSULTORIA ÚNICA ESPECIALITZ...,España,Barcelona -,NaN,Contrato


In [19]:
clean.isna().sum()

url                     0
fecha_publicacion       0
fecha_limite            6
titulo                  0
empresa              7241
ocupacion             735
educacion               0
descripcion             0
pais                    0
region                115
duracion_jornada     7305
tipo_contrato           2
dtype: int64

Hay que borrar filas de fecha nula

In [20]:
clean.dropna(subset="fecha_limite", inplace=True)

In [21]:
def cleanDash(region):
    if type(region) == str:
        if region[-1] == "-":
            region = region[:-1]
        return region.strip()
    return region

clean["region"] = clean["region"].apply(cleanDash)

In [22]:
clean.head()

,url,fecha_publicacion,fecha_limite,titulo,empresa,ocupacion,educacion,descripcion,pais,region,duracion_jornada,tipo_contrato
0,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,31/10/2025,"CONDUCTORES DE CAMIÓN, EN GENERAL",NaN,Conductor de vehículo de carga/conductora de v...,Contrato,REQUIERE ESTAR EN POSESIÓN DEL CAP. LA EMPRESA...,España,Palencia,NaN,Contrato
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,10/10/2025,WORK/LOCATION,NaN,Empleado administrativo de archivos/empleada a...,Contrato,ES BUSCA UNA PERSONA AMB ESTUDIS I EXPERIèNCIA...,España,Tarragona,NaN,Contrato
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,24/10/2025,WORK/LOCATION,NaN,Operario especializado en engranajes de automo...,Contrato,CERQUEM PROFESSIONALS AMB EXPERIèNCIA EN MUNTA...,España,Girona,NaN,Contrato
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,13/10/2025,WORK/LOCATION,NaN,Empleado del hogar/empleada del hogar,Contrato,IMPORTANTE EMPRESA UBICADA EN MERCABARNA PRECI...,España,Barcelona,NaN,Contrato
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,20/10/2025,WORK/LOCATION,NaN,Conductor de servicios de transporte de enferm...,Determinado,REQUISITS: ? TITULACIÓ D? ESPECIALISTA EN FARM...,España,Barcelona,NaN,Determinado


### Columnas sin informacion

In [23]:
clean.set_index("url", inplace= True)
clean.drop(["educacion", "duracion_jornada"], inplace= True, axis="columns")
clean.shape

(7299, 9)

In [24]:
clean.head()

,fecha_publicacion,fecha_limite,titulo,empresa,ocupacion,descripcion,pais,region,tipo_contrato
url,,,,,,,,,
https://europa.eu/eures/portal/jv-se/jv-details/NzE4NDAzMCAxMDI?lang=es,10/10/2025,31/10/2025,"CONDUCTORES DE CAMIÓN, EN GENERAL",NaN,Conductor de vehículo de carga/conductora de v...,REQUIERE ESTAR EN POSESIÓN DEL CAP. LA EMPRESA...,España,Palencia,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyMDEwMCAxMDI?lang=es,10/10/2025,10/10/2025,WORK/LOCATION,NaN,Empleado administrativo de archivos/empleada a...,ES BUSCA UNA PERSONA AMB ESTUDIS I EXPERIèNCIA...,España,Tarragona,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyNzY1MCAxMDI?lang=es,10/10/2025,24/10/2025,WORK/LOCATION,NaN,Operario especializado en engranajes de automo...,CERQUEM PROFESSIONALS AMB EXPERIèNCIA EN MUNTA...,España,Girona,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyMDg4OSAxMDI?lang=es,10/10/2025,13/10/2025,WORK/LOCATION,NaN,Empleado del hogar/empleada del hogar,IMPORTANTE EMPRESA UBICADA EN MERCABARNA PRECI...,España,Barcelona,Contrato
https://europa.eu/eures/portal/jv-se/jv-details/NzIyNDgzOSAxMDI?lang=es,10/10/2025,20/10/2025,WORK/LOCATION,NaN,Conductor de servicios de transporte de enferm...,REQUISITS: ? TITULACIÓ D? ESPECIALISTA EN FARM...,España,Barcelona,Determinado


In [51]:
clean.to_csv("ofertas.csv")

## Análisis temporalidad

In [25]:
clean = pd.read_csv("ofertas.csv")
clean.fecha_limite.unique()

array(['31/10/2025', '10/10/2025', '24/10/2025', '13/10/2025',
       '20/10/2025', '21/10/2025', '15/10/2025', '23/01/2026',
       '22/10/2025', '30/10/2025', '16/10/2025', '15/01/2026',
       '18/10/2025', '23/10/2025', '31/12/2025', '14/10/2025',
       '29/10/2025', '25/10/2025', '20/12/2025', '11/10/2025',
       '11/11/2025', '4/11/2025', '17/10/2025', '30/12/2025', '7/11/2025',
       '8/11/2025', '25/02/2026', '25/11/2025', '3/11/2025', '2/12/2025',
       '22/11/2025', '14/12/2025', '26/11/2025', '1/12/2025',
       '17/11/2025', '12/10/2025', '28/10/2025', '18/11/2025',
       '12/11/2025', '2/11/2025', '8/12/2025', '1/11/2025', '27/10/2025',
       '10/12/2025', '18/12/2025', '5/11/2025', '30/11/2025',
       '29/11/2025', '19/10/2025', '26/10/2025', '17/12/2025',
       '21/12/2025', '13/11/2025', '11/12/2025', '28/11/2025',
       '19/12/2025', '15/12/2025', '3/12/2025', '21/11/2025',
       '10/11/2025', '9/12/2025', '5/12/2025', '24/11/2025', '27/11/2025',
       '15/1

In [26]:
# Convertimos tu array a fechas
fechas = pd.to_datetime(clean.fecha_limite.unique(), dayfirst=True)

# Rango completo entre la fecha mínima y máxima
rango_completo = pd.date_range(start=fechas.min(), end=fechas.max())

# Verificamos si hay fechas faltantes
faltantes = rango_completo.difference(fechas)

print(f"Faltan {len(faltantes)} días en el rango.")


Faltan 284 días en el rango.


In [27]:
# Agrupamos por semana (usamos el número de semana ISO)
semanas_presentes = fechas.isocalendar().week.unique()

# Creamos el rango completo de semanas entre la primera y última fecha
rango_completo = pd.date_range(start=fechas.min(), end=fechas.max(), freq='W-MON')
semanas_completas = rango_completo.isocalendar().week.unique()

# Comparamos
faltantes = set(semanas_completas) - set(semanas_presentes)

print(f"Total semanas en el rango: {len(semanas_completas)}")
print(f"Semanas faltantes: {len(faltantes)}")

Total semanas en el rango: 52
Semanas faltantes: 31


In [28]:
# Extraer año y mes únicos presentes
meses_presentes = fechas.to_period('M').unique()

# Crear rango completo de meses entre la fecha mínima y máxima
rango_meses = pd.period_range(start=fechas.min(), end=fechas.max(), freq='M')

# Comparar
faltantes = set(rango_meses) - set(meses_presentes)

print(f"Meses presentes: {len(meses_presentes)}")
print(f"Meses faltantes: {len(faltantes)}")


Meses presentes: 7
Meses faltantes: 6


In [29]:
sorted(fechas)

[Timestamp('2025-10-10 00:00:00'),
 Timestamp('2025-10-11 00:00:00'),
 Timestamp('2025-10-12 00:00:00'),
 Timestamp('2025-10-13 00:00:00'),
 Timestamp('2025-10-14 00:00:00'),
 Timestamp('2025-10-15 00:00:00'),
 Timestamp('2025-10-16 00:00:00'),
 Timestamp('2025-10-17 00:00:00'),
 Timestamp('2025-10-18 00:00:00'),
 Timestamp('2025-10-19 00:00:00'),
 Timestamp('2025-10-20 00:00:00'),
 Timestamp('2025-10-21 00:00:00'),
 Timestamp('2025-10-22 00:00:00'),
 Timestamp('2025-10-23 00:00:00'),
 Timestamp('2025-10-24 00:00:00'),
 Timestamp('2025-10-25 00:00:00'),
 Timestamp('2025-10-26 00:00:00'),
 Timestamp('2025-10-27 00:00:00'),
 Timestamp('2025-10-28 00:00:00'),
 Timestamp('2025-10-29 00:00:00'),
 Timestamp('2025-10-30 00:00:00'),
 Timestamp('2025-10-31 00:00:00'),
 Timestamp('2025-11-01 00:00:00'),
 Timestamp('2025-11-02 00:00:00'),
 Timestamp('2025-11-03 00:00:00'),
 Timestamp('2025-11-04 00:00:00'),
 Timestamp('2025-11-05 00:00:00'),
 Timestamp('2025-11-06 00:00:00'),
 Timestamp('2025-11-

Están todos los días de octubre y noviembre, casi todos los de diciembre e inicio de enero.

El resto de fechas son más dispersas

## Análisis títulos repetidos

In [30]:
clean = pd.read_csv("ofertas.csv")
clean["titulo"].value_counts()

titulo
WORK/LOCATION                                              3562
PERSONAL DE LIMPIEZA O LIMPIADORES EN GENERAL               110
CONDUCTORES DE CAMIÓN, EN GENERAL                            82
COCINEROS, EN GENERAL                                        68
ENFERMEROS DE CUIDADOS GENERALES                             62
                                                           ... 
CAMARERO/A (REF.5904)                                         1
MOZO ALMACEN//REPARTIDOR                                      1
MONTADORES DE GRANDES ESTRUCTURAS METÁLICAS, EN GENERAL       1
AUXILIARES DE ENFERMERIA/GERIATRIA                            1
AUXILIAR DE AYUDA A DOMICILIO - PILOÑA                        1
Name: count, Length: 1923, dtype: int64

In [31]:
descriptiveName = clean[clean["titulo"] != "WORK/LOCATION"]
descriptiveName.shape

(3737, 10)

In [32]:
fechas = pd.to_datetime(descriptiveName.fecha_limite.unique(), dayfirst=True)

In [33]:
sorted(fechas)

[Timestamp('2025-10-10 00:00:00'),
 Timestamp('2025-10-11 00:00:00'),
 Timestamp('2025-10-12 00:00:00'),
 Timestamp('2025-10-13 00:00:00'),
 Timestamp('2025-10-14 00:00:00'),
 Timestamp('2025-10-15 00:00:00'),
 Timestamp('2025-10-16 00:00:00'),
 Timestamp('2025-10-17 00:00:00'),
 Timestamp('2025-10-18 00:00:00'),
 Timestamp('2025-10-19 00:00:00'),
 Timestamp('2025-10-20 00:00:00'),
 Timestamp('2025-10-21 00:00:00'),
 Timestamp('2025-10-22 00:00:00'),
 Timestamp('2025-10-23 00:00:00'),
 Timestamp('2025-10-24 00:00:00'),
 Timestamp('2025-10-25 00:00:00'),
 Timestamp('2025-10-26 00:00:00'),
 Timestamp('2025-10-27 00:00:00'),
 Timestamp('2025-10-28 00:00:00'),
 Timestamp('2025-10-29 00:00:00'),
 Timestamp('2025-10-30 00:00:00'),
 Timestamp('2025-10-31 00:00:00'),
 Timestamp('2025-11-01 00:00:00'),
 Timestamp('2025-11-02 00:00:00'),
 Timestamp('2025-11-03 00:00:00'),
 Timestamp('2025-11-04 00:00:00'),
 Timestamp('2025-11-05 00:00:00'),
 Timestamp('2025-11-06 00:00:00'),
 Timestamp('2025-11-

In [34]:
descriptiveName.set_index("url", inplace= True)
descriptiveName.shape

(3737, 9)

In [35]:
descriptiveName.to_csv("ofertasDescriptive.csv")

In [36]:
set(pd.to_datetime(clean.fecha_limite.unique(), dayfirst=True)) - set(pd.to_datetime(descriptiveName.fecha_limite.unique(), dayfirst=True))

{Timestamp('2026-01-02 00:00:00')}

In [37]:
clean[clean["titulo"] == "WORK/LOCATION"]

,url,fecha_publicacion,fecha_limite,titulo,empresa,ocupacion,descripcion,pais,region,tipo_contrato
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,10/10/2025,WORK/LOCATION,NaN,Empleado administrativo de archivos/empleada a...,ES BUSCA UNA PERSONA AMB ESTUDIS I EXPERIèNCIA...,España,Tarragona,Contrato
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,24/10/2025,WORK/LOCATION,NaN,Operario especializado en engranajes de automo...,CERQUEM PROFESSIONALS AMB EXPERIèNCIA EN MUNTA...,España,Girona,Contrato
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,13/10/2025,WORK/LOCATION,NaN,Empleado del hogar/empleada del hogar,IMPORTANTE EMPRESA UBICADA EN MERCABARNA PRECI...,España,Barcelona,Contrato
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,20/10/2025,WORK/LOCATION,NaN,Conductor de servicios de transporte de enferm...,REQUISITS: ? TITULACIÓ D? ESPECIALISTA EN FARM...,España,Barcelona,Determinado
6,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,20/10/2025,WORK/LOCATION,NaN,Conductor de servicios de transporte de enferm...,T?AGRADARIA TREBALLAR A LA FUNDACIÓ LAFACT ?LA...,España,Barcelona,Contrato
...,...,...,...,...,...,...,...,...,...,...
7292,https://europa.eu/eures/portal/jv-se/jv-detail...,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Empleado del hogar/empleada del hogar,CONTRACTE DE SUBSTITUCIÓ PER MATERNITAT PER UN...,España,Tarragona,Determinado
7293,https://europa.eu/eures/portal/jv-se/jv-detail...,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Mozo de almacén/moza de almacén,EMPRESA DE SERVICIOS AUXILIARES NECESITA PERSO...,España,Barcelona,Contrato
7294,https://europa.eu/eures/portal/jv-se/jv-detail...,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Inspector de carga y flete/inspectora de carga...,"ACTUALMENTE, DESDE LA DELEGACIÓN DE TARRAGONA ...",España,Barcelona,Determinado
7295,https://europa.eu/eures/portal/jv-se/jv-detail...,11/10/2025,20/11/2025,WORK/LOCATION,NaN,Profesor de autoescuela/profesora de autoescuela,"NDICE CONSULTORÍA Y FORMACIÓN, SOMOS UNA CONSU...",España,Barcelona,Autónomo


In [38]:
clean["ocupacion"].value_counts()

ocupacion
Empleado del hogar/empleada del hogar                    306
Mozo de almacén/moza de almacén                          251
Barrendero/barrendera                                    248
Empleado de oficina/empleada de oficina                  214
Auxiliar de enfermería                                   190
                                                        ... 
Técnico agrícola/técnica agrícola                          1
técnico en geología/técnica en geología                    1
Agente de conservación forestal                            1
supervisor de destilería/supervisora de destilería         1
ingeniero técnico agrícola/ingeniera técnica agrícola      1
Name: count, Length: 463, dtype: int64

Pese a que no todos las ofertas tienen título, si tienen ocupación, por lo que se puede usar para clasificar aparte del título.


Aun así, lo guardamos en 2 csv:
- `ofertas.csv`: Todas las 7299 ofertas, 3562 sin título
- `ofertasDescriptive.csv`: Solo las 3737 ofertas con nombre descriptivo

In [39]:
clean.isna().sum()  / 7299 *100 

url                   0.000000
fecha_publicacion     0.000000
fecha_limite          0.000000
titulo                0.000000
empresa              99.205371
ocupacion            10.056172
descripcion           0.000000
pais                  0.000000
region                1.561858
tipo_contrato         0.027401
dtype: float64

## Anonimización de la base de datos

Para cumplir con los principios FAIR es necesario anonimizar la base de datos, que en nuestro caso corresponde con eliminar los correos electrónicos y números de teléfono presentes en los datos.

In [61]:
clean = pd.read_csv("ofertas.csv")
clean["titulo"].value_counts()

titulo
WORK/LOCATION                                              3562
PERSONAL DE LIMPIEZA O LIMPIADORES EN GENERAL               110
CONDUCTORES DE CAMIÓN, EN GENERAL                            82
COCINEROS, EN GENERAL                                        68
ENFERMEROS DE CUIDADOS GENERALES                             62
                                                           ... 
CAMARERO/A (REF.5904)                                         1
MOZO ALMACEN//REPARTIDOR                                      1
MONTADORES DE GRANDES ESTRUCTURAS METÁLICAS, EN GENERAL       1
AUXILIARES DE ENFERMERIA/GERIATRIA                            1
AUXILIAR DE AYUDA A DOMICILIO - PILOÑA                        1
Name: count, Length: 1923, dtype: int64

Previo a estas operaciones, vamos a juntar las columnas descripcion, pais, region, tipo_contrato en una sola, ya que al ser archivos csv, la existencia de comas supone un error en los datos, pues como muchas descripciones contienen comas, se han separado los datos en distinas columnas.

Por comodidad vamos a juntarlas todas en una y considerarla como una sola

In [62]:
# Columnas a combinar 
cols_to_merge = ["descripcion", "pais", "region", "tipo_contrato"]

# Combina el contenido
clean["descripcion"] = clean[cols_to_merge].fillna("").agg(" ".join, axis=1)

# elimina las columnas originales
clean = clean.drop(columns=["pais", "region", "tipo_contrato"])


In [ ]:
# expresiones regulares
email_regex = re.compile(
    r"""[a-zA-Z0-9.!#$%&'*+/=?^_`{|}~-]+@
        [a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+""",
    flags=re.VERBOSE | re.IGNORECASE
)

phone_regex_spain = re.compile(
    r"(?:\+34\s?)?(?:6|7|8|9)\d{8}\b"      
)

# buscar coincidencias en el DataFrame
def find_pii(df: pd.DataFrame, target_col: str = "descripcion"):
    """
    Busca correos y teléfonos del DataFrame.
    """
    hits = []
    ser = df[target_col].astype(str).fillna("")

    # emails
    mask_email = ser.str.contains(email_regex, regex=True, na=False)
    for idx in df.index[mask_email]:
        matches = re.findall(email_regex, df.at[idx, target_col])
        for match in matches:
            hits.append((idx, target_col, "email", match))

    # teléfonos
    mask_phone_spain = ser.str.contains(phone_regex_spain, regex=True, na=False)
    for idx in df.index[mask_phone_spain]:
        matches = re.findall(phone_regex_spain, df.at[idx, target_col])
        for match in matches:
            hits.append((idx, target_col, "phone_spain", match))

    # devolver resultados como DataFrame
    result = pd.DataFrame(hits, columns=["index", "column", "type", "match"])
    return result

found = find_pii(clean)

In [66]:
# función para anonimizar el DataFrame
def anonymize_df(df: pd.DataFrame, target_col: str = "descripcion", replace_email: str = "[EMAIL]", replace_phone: str = "[PHONE]"):
    """
    Anonimiza  la columna indicada reemplazando correos y teléfonos por etiquetas.
    """
    df2 = df.copy()

    if pd.api.types.is_string_dtype(df2[target_col]) or pd.api.types.is_object_dtype(df2[target_col]):
        df2[target_col] = df2[target_col].astype(str)
        #  correos
        df2[target_col] = df2[target_col].str.replace(email_regex, replace_email, regex=True)
        #  teléfonos
        df2[target_col] = df2[target_col].str.replace(phone_regex_spain, replace_phone, regex=True)
    return df2


clean=anonymize_df(clean)

In [ ]:
clean.to_csv("ofertasAnonimizado.csv")